# Ensemble
* 모델 앙상블

In [48]:
import numpy as np
import pandas as pd

print("로딩 중: vit_final_preds.npy, effnet_final_preds.npy, conv_final_preds.npy")
vit_preds = np.load("npy/vit_final_preds.npy")
eff_b4_preds = np.load("npy/eff_b4_final_preds.npy")
eff_b5_preds = np.load("npy/eff_b5_final_preds.npy")
conv_preds = np.load("npy/conv_final_preds.npy")

# --- 최종 앙상블 ---
print("20:30:30:20 가중치로 앙상블 중...")
final_avg_preds = (eff_b4_preds * 0.15) + (eff_b5_preds * 0.15) + (vit_preds * 0.5) + (conv_preds * 0.2)

# 최종 라벨
final_labels = np.argmax(final_avg_preds, axis=1)

# 제출 파일 생성
tst_df = pd.read_csv("datasets_fin/sample_submission.csv")
submission_df = tst_df.copy()
submission_df['target'] = final_labels

submission_df.to_csv("submission/submission_ENSEMBLE_effb415_effb515_vit50_conv20.csv", index=False)
print("✅ 최종 앙상블 제출 파일 'submission_ENSEMBLE_effb415_effb515_vit50_conv20.csv' 생성 완료!")

로딩 중: vit_final_preds.npy, effnet_final_preds.npy, conv_final_preds.npy
20:30:30:20 가중치로 앙상블 중...
✅ 최종 앙상블 제출 파일 'submission_ENSEMBLE_effb415_effb515_vit50_conv20.csv' 생성 완료!


# OCR
* easyocr

In [2]:
!pip install easyocr
!pip install scikit-learn

  Obtaining dependency information for easyocr from https://files.pythonhosted.org/packages/bb/84/4a2cab0e6adde6a85e7ba543862e5fc0250c51f3ac721a078a55cdcff250/easyocr-1.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for python-bidi from https://files.pythonhosted.org/packages/d3/30/0753601fdad405e806c89cfa9603ff75241f8c7196cfe2cb37c43e34cdbd/python_bidi-0.6.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for Shapely from https://files.pythonhosted.org/packages/50/8a/0ab1f7433a2a85d9e9aea5b1fbb333f3b09b309e7817309250b4b7b2cc7a/shapely-2.1.2-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
  Obtaining dependency information for pyclipper from https://files.pythonhosted.org/packages/84/a4/3e304f6c0d000382cd54d4a1e5f0d8fc28e1ae97413a2ec1016a7b840319/pyclipper-1.3.0.post6-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata
  Obtaining dependency information for ninja from https://fi

In [3]:
import easyocr
import pandas as pd
import numpy as np
import os
import cv2
from tqdm import tqdm

# 1. EasyOCR 리더기 초기화 (한국어, 영어)
reader = easyocr.Reader(['ko', 'en'])

# 2. 텍스트를 추출할 원본 CSV 로드
try:
    all_df = pd.read_csv("datasets_fin/train.csv") # (1570개)
    tst_df = pd.read_csv("datasets_fin/sample_submission.csv") # (3140개)
except FileNotFoundError:
    print("오류: train.csv 또는 sample_submission.csv 파일을 찾을 수 없습니다.")

ocr_results = []
img_dir = "datasets_fin/train"
print(f"--- 학습 이미지 (train.csv) {len(all_df)}개 텍스트 추출 중 ---")

# 3. 학습 이미지(1570개)에서 텍스트 추출
for idx, row in tqdm(all_df.iterrows(), total=len(all_df)):
    img_path = os.path.join(img_dir, row['ID'])
    try:
        # OCR 실행
        result = reader.readtext(img_path, detail=0) # detail=0은 텍스트만 추출
        extracted_text = ' '.join(result) # 추출된 텍스트 조각들을 하나로 합침
        ocr_results.append({'ID': row['ID'], 'target': row['target'], 'text': extracted_text})
    except Exception as e:
        print(f"오류 발생 ({row['ID']}): {e}")
        ocr_results.append({'ID': row['ID'], 'target': row['target'], 'text': ''}) # 실패 시 빈 텍스트

# 4. train_ocr.csv로 저장
ocr_train_df = pd.DataFrame(ocr_results)
ocr_train_df.to_csv("datasets_fin/train_ocr.csv", index=False)
print("✅ 'train_ocr.csv' 저장 완료!")


# 5. 테스트 이미지(3140개)에서 텍스트 추출
ocr_results = []
img_dir = "datasets_fin/test"
print(f"--- 테스트 이미지 (tst_df) {len(tst_df)}개 텍스트 추출 중 ---")

for idx, row in tqdm(tst_df.iterrows(), total=len(tst_df)):
    img_path = os.path.join(img_dir, row['ID'])
    try:
        result = reader.readtext(img_path, detail=0)
        extracted_text = ' '.join(result)
        ocr_results.append({'ID': row['ID'], 'target': row['target'], 'text': extracted_text})
    except Exception as e:
        print(f"오류 발생 ({row['ID']}): {e}")
        ocr_results.append({'ID': row['ID'], 'target': row['target'], 'text': ''})

# 6. test_ocr.csv로 저장
ocr_test_df = pd.DataFrame(ocr_results)
ocr_test_df.to_csv("datasets_fin/test_ocr.csv", index=False)
print("✅ 'test_ocr.csv' 저장 완료!")

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete--- 학습 이미지 (train.csv) 1570개 텍스트 추출 중 ---


100%|██████████| 1570/1570 [16:21<00:00,  1.60it/s]


✅ 'train_ocr.csv' 저장 완료!
--- 테스트 이미지 (tst_df) 3140개 텍스트 추출 중 ---


100%|██████████| 3140/3140 [08:49<00:00,  5.93it/s]

✅ 'test_ocr.csv' 저장 완료!


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

print("--- 텍스트 모델(Tfidf + LR) 학습 및 검증 시작 ---")

# 1. 텍스트 데이터 로드
try:
    ocr_train_df = pd.read_csv("datasets_fin/train_ocr.csv")
    # 'text' 컬럼이 비어있는(NaN) 경우, 빈 문자열로 채웁니다.
    ocr_train_df['text'] = ocr_train_df['text'].fillna('')
except FileNotFoundError:
    print("오류: 2단계에서 생성된 'train_ocr.csv' 파일을 찾을 수 없습니다.")

# 2. [✅ 핵심] 이미지 모델과 *똑같은* '미니 리더보드' 분리
# (random_state=42를 사용하여 'holdout_df'가 이미지 모델과 동일하게 분리됨)
text_train_df, text_holdout_df = train_test_split(
    ocr_train_df, 
    test_size=0.2,    
    random_state=42,  # <--- 이미지 모델과 동일한 random_state
    stratify=ocr_train_df['target'] 
)
print(f"텍스트 학습: {len(text_train_df)}개, 텍스트 홀드아웃: {len(text_holdout_df)}개")

# 3. 텍스트 특징 추출기(Tfidf) 정의 및 학습
# (단어/글자 단위(ngram)로 특징을 잡습니다)
vectorizer = TfidfVectorizer(
    analyzer='char_wb', # 단어(word boundary) 사이의 글자(char)
    ngram_range=(2, 5), # 2글자 ~ 5글자 조합을 모두 특징으로 사용
    max_features=20000  # 최대 특징 개수
)

# 학습 텍스트(1256개)로 Tfidf 학습
X_train_text = vectorizer.fit_transform(text_train_df['text'])
y_train_text = text_train_df['target']
print("Tfidf 벡터화 완료.")

# 4. 텍스트 분류기(LogisticRegression) 학습
text_model = LogisticRegression(solver='liblinear', random_state=42)
text_model.fit(X_train_text, y_train_text)
print("텍스트 모델 학습 완료.")

# 5. [✅] 텍스트 모델 '미니 리더보드' 점수 확인
X_holdout_text = vectorizer.transform(text_holdout_df['text'])
y_holdout_true = text_holdout_df['target']
y_holdout_pred_text = text_model.predict(X_holdout_text)

text_f1_score = f1_score(y_holdout_true, y_holdout_pred_text, average='macro')
print(f"--- 텍스트 모델 '미니 리더보드' F1 점수: {text_f1_score:.4f} ---")

--- 텍스트 모델(Tfidf + LR) 학습 및 검증 시작 ---
텍스트 학습: 1256개, 텍스트 홀드아웃: 314개
Tfidf 벡터화 완료.
텍스트 모델 학습 완료.
--- 텍스트 모델 '미니 리더보드' F1 점수: 0.8866 ---


In [6]:
import numpy as np
import pandas as pd

print("--- 최종 4-모델 앙상블 (Image*3 + Text*1) 시작 ---")

# --- (A) 텍스트 모델 예측 확률 생성 ---
print("1. 텍스트 모델(.npy) 생성 중...")
try:
    ocr_test_df = pd.read_csv("datasets_fin/test_ocr.csv")
    ocr_test_df['text'] = ocr_test_df['text'].fillna('')
    
    # [✅] '진짜 테스트 셋' 텍스트를 벡터화
    X_test_text = vectorizer.transform(ocr_test_df['text'])
    
    # [✅] '진짜 테스트 셋'의 확률(predict_proba) 예측
    text_model_probs = text_model.predict_proba(X_test_text) # (3140, 17)
    
    # 4번째 앙상블 멤버로 저장
    np.save("npy/text_final_preds.npy", text_model_probs)
    print("✅ 'text_final_preds.npy' 저장 완료.")

except Exception as e:
    print(f"오류: 텍스트 모델 예측 중 오류 발생: {e}")

--- 최종 4-모델 앙상블 (Image*3 + Text*1) 시작 ---
1. 텍스트 모델(.npy) 생성 중...
✅ 'text_final_preds.npy' 저장 완료.


In [8]:
# --- (B) 3-모델 이미지 앙상블 + 텍스트 앙상블 ---
print("2. 이미지 3개 + 텍스트 1개 앙상블 블렌딩...")
try:
    # 1. 기존 이미지 모델 3개의 확률 로드
    vit_preds = np.load("npy/vit_final_preds.npy")
    effnet_preds = np.load("npy/effnet_final_preds.npy")
    convnext_preds = np.load("npy/conv_final_preds.npy")
    
    # 2. 방금 생성한 텍스트 모델 확률 로드
    text_preds = np.load("npy/text_final_preds.npy")

    # 3. [✅ 핵심] 가중치 조절
    # 예시: 0.9069점을 만든 이미지 앙상블(가중치 70%) + 텍스트 모델(가중치 30%)
    
    # 3-1. 이미지 앙상블 (0.9069점짜리)
    image_ensemble_probs = (convnext_preds * 0.4) + (effnet_preds * 0.4) + (vit_preds * 0.2)
    
    # 3-2. 이미지(70%) + 텍스트(30%) 최종 앙상블
    final_avg_preds = (image_ensemble_probs * 0.95) + (text_preds * 0.05)
    
    # (팁: 이 0.7, 0.3 가중치를 0.6:0.4, 0.8:0.2 등으로 조절하며 최고 점수를 찾아야 합니다)

    # 4. 최종 라벨
    final_labels = np.argmax(final_avg_preds, axis=1)

    # 5. 제출 파일 생성
    tst_df = pd.read_csv("datasets_fin/sample_submission.csv")
    submission_df = tst_df.copy()
    submission_df['target'] = final_labels

    submission_df.to_csv("submission/submission_MULTIMODAL_FINAL2.csv", index=False)
    print("✅ 최종 멀티모달 앙상블 제출 파일 'submission_MULTIMODAL_FINAL2.csv' 생성 완료!")

except FileNotFoundError:
    print("오류: .npy 파일 (vit, effnet, convnext) 중 일부를 찾을 수 없습니다.")

2. 이미지 3개 + 텍스트 1개 앙상블 블렌딩...
✅ 최종 멀티모달 앙상블 제출 파일 'submission_MULTIMODAL_FINAL2.csv' 생성 완료!


* KLUE

In [49]:
!pip install transformers[sentencepiece]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
import torch
from torch.utils.data import Dataset, DataLoader
# [✅ 교체] BertTokenizerFast 대신, 'AutoTokenizer'를 사용합니다
from transformers import AutoTokenizer, AutoModelForSequenceClassification 
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# --- (A) KLUE 모델 및 토크나이저 정의 (표준 방식) ---
KLUE_MODEL_NAME = 'klue/roberta-base'

# 'AutoTokenizer'가 KLUE 모델을 자동으로 인식하고 로드합니다
tokenizer = AutoTokenizer.from_pretrained(KLUE_MODEL_NAME) 

# --- (B) KoBERT용 데이터셋 클래스 (이름만 KoBERT, 내용은 동일) ---
class KoBERTDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['text'].values
        self.targets = dataframe['target'].values
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        target = self.targets[index]

        # KLUE 토크나이저로 텍스트 인코딩 (KoBERT와 동일)
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True, 
            max_length=self.max_len,
            padding='max_length', 
            truncation=True, 
            return_token_type_ids=False,
            return_attention_mask=True, 
            return_tensors='pt' 
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

print("✅ KLUE/roberta-base 토크나이저 및 데이터셋 클래스 정의 완료.")

✅ KLUE/roberta-base 토크나이저 및 데이터셋 클래스 정의 완료.


In [52]:
from sklearn.metrics import f1_score
from tqdm import tqdm

# --- (A) 하이퍼파라미터 (KoBERT와 동일) ---
MAX_LEN = 128      
BATCH_SIZE = 16    
EPOCHS = 20         
LR = 3e-5          
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- (B) 데이터로더 생성 (KoBERT 2단계에서 만든 변수 사용) ---
# (text_train_df, text_holdout_df 변수가 살아있어야 합니다)
train_dataset = KoBERTDataset(text_train_df, tokenizer, MAX_LEN)
holdout_dataset = KoBERTDataset(text_holdout_df, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
holdout_loader = DataLoader(holdout_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- (C) 모델 및 옵티마이저 ---
# [✅ 교체] 'AutoModel'이 KLUE 모델을 자동으로 로드합니다
model = AutoModelForSequenceClassification.from_pretrained(KLUE_MODEL_NAME, num_labels=17).to(device)
optimizer = AdamW(model.parameters(), lr=LR)
loss_fn = torch.nn.CrossEntropyLoss().to(device)

best_val_f1 = 0.0

print("--- 텍스트 모델(KLUE) '미니 리더보드' 학습 시작 ---")

for epoch in range(EPOCHS):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{EPOCHS} [Train]")
    for batch in pbar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        pbar.set_description(f"Epoch {epoch + 1}/{EPOCHS} [Train] Loss: {loss.item():.4f}")

    # --- [✅ 핵심 검증] '미니 리더보드' F1 점수 확인 ---
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for batch in tqdm(holdout_loader, desc=f"Epoch {epoch + 1}/{EPOCHS} [Valid]"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    val_f1 = f1_score(all_targets, all_preds, average='macro')
    print(f"--- 텍스트 모델(KLUE) '미니 리더보드' F1 점수: {val_f1:.4f} ---")

    # [✅] 0.91+ 점을 넘어야 앙상블에 기여 가능!
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "model/klue_best_model.pth")
        print(f"⭐️ KLUE Best F1 Score updated to {val_f1:.4f}. Model saved!")

print(f"--- KLUE 학습 완료! 최종 '미니 리더보드' 점수: {best_val_f1:.4f} ---")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- 텍스트 모델(KLUE) '미니 리더보드' 학습 시작 ---


Epoch 1/20 [Valid]: 100%|██████████| 20/20 [00:00<00:00, 33.61it/s]


--- 텍스트 모델(KLUE) '미니 리더보드' F1 점수: 0.5084 ---
⭐️ KLUE Best F1 Score updated to 0.5084. Model saved!


Epoch 2/20 [Valid]: 100%|██████████| 20/20 [00:00<00:00, 33.43it/s]


...

--- 텍스트 모델(KLUE) '미니 리더보드' F1 점수: 0.9276 ---


Epoch 19/20 [Valid]: 100%|██████████| 20/20 [00:00<00:00, 33.39it/s]


--- 텍스트 모델(KLUE) '미니 리더보드' F1 점수: 0.9313 ---


Epoch 20/20 [Valid]: 100%|██████████| 20/20 [00:00<00:00, 33.27it/s]

--- 텍스트 모델(KLUE) '미니 리더보드' F1 점수: 0.9445 ---
--- KLUE 학습 완료! 최종 '미니 리더보드' 점수: 0.9509 ---


In [53]:
import numpy as np
import pandas as pd
from tqdm import tqdm

print("--- 텍스트 모델(KLUE) '진짜 테스트 데이터' 추론 시작 (NPY 생성) ---")

# 1. 테스트 텍스트 로드 (easyocr로 생성한 파일)
ocr_test_df = pd.read_csv("datasets_fin/test_ocr.csv").fillna('')
test_dataset = KoBERTDataset(ocr_test_df, tokenizer, MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE * 2, shuffle=False) # 추론은 배치 2배

# 2. 저장된 Best KLUE 모델 로드
# [✅ KLUE] AutoModel로 로드
model = AutoModelForSequenceClassification.from_pretrained(KLUE_MODEL_NAME, num_labels=17).to(device)
model.load_state_dict(torch.load("model/klue_best_model.pth"))
model.eval()

# 3. 추론 및 확률 추출
all_probs = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="KLUE TTA 추론 중"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # [✅] 로짓을 확률(softmax)로 변환
        probs = torch.softmax(logits, dim=1) 
        all_probs.append(probs.cpu().numpy())

# 4. .npy 파일로 저장
klue_final_preds_array = np.concatenate(all_probs, axis=0) # (3140, 17)
np.save("npy/klue_final_preds.npy", klue_final_preds_array)
print("✅ KLUE (Test Data) 확률 저장 완료! (klue_final_preds.npy)")

--- 텍스트 모델(KLUE) '진짜 테스트 데이터' 추론 시작 (NPY 생성) ---


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
KLUE TTA 추론 중: 100%|██████████| 99/99 [00:05<00:00, 19.54it/s]

✅ KLUE (Test Data) 확률 저장 완료! (klue_final_preds.npy)


In [58]:
import numpy as np
import pandas as pd

print("--- 최종 4-모델 앙상블 (Image*3 + KLUE*1) 시작 ---")

try:
    # 1. 기존 이미지 모델 3개의 확률 로드
    vit_preds = np.load("npy/vit_final_preds.npy")
    eff_b5_preds = np.load("npy/eff_b5_final_preds.npy")
    convnext_preds = np.load("npy/conv_final_preds.npy")
    
    # 2. 방금 생성한 KLUE 확률 로드
    text_preds = np.load("npy/klue_final_preds.npy")

    # 3. [✅ 핵심] 4-모델 가중치
    # 이미지 앙상블(0.9069점)과 KLUE(0.9067점)가 거의 동일한 수준이므로,
    # 80:20 또는 70:30 가중치를 시도해볼 가치가 있습니다.
    
    # 3-1. 이미지 앙상블 (0.9069점짜리)
    image_ensemble_probs = (eff_b5_preds * 0.2) + (vit_preds * 0.5) + (convnext_preds * 0.3)
    
    # 3-2. 이미지(80%) + KLUE(20%) 최종 앙상블
    final_avg_preds = (image_ensemble_probs * 1.0) + (text_preds * 0)

    # 4. 최종 라벨
    final_labels = np.argmax(final_avg_preds, axis=1)

    # 5. 제출 파일 생성
    tst_df = pd.read_csv("datasets_fin/sample_submission.csv")
    submission_df = tst_df.copy()
    submission_df['target'] = final_labels

    submission_df.to_csv("submission/submission_MULTIMODAL_KLUE_100_0.csv", index=False)
    print("✅ 최종 멀티모달(KLUE 90:10) 앙상블 제출 파일 'submission_MULTIMODAL_KLUE_100_0.csv' 생성 완료!")

except FileNotFoundError:
    print("오류: .npy 파일 (vit, effnet, convnext, klue) 중 일부를 찾을 수 없습니다.")

--- 최종 4-모델 앙상블 (Image*3 + KLUE*1) 시작 ---
✅ 최종 멀티모달(KLUE 90:10) 앙상블 제출 파일 'submission_MULTIMODAL_KLUE_100_0.csv' 생성 완료!


# 의사 라벨링 (Pseudo-Labeling)

In [36]:
import numpy as np
import pandas as pd
import os
import shutil # (파일 복사를 위해 import)

print("--- 1단계: '가짜 정답(Pseudo-Label)' 생성 시작 ---")

# 1. 3-모델 .npy 파일 로드
try:
    vit_preds = np.load("npy/vit_final_preds.npy")
    effnet_preds = np.load("npy/effnet_final_preds.npy")
    convnext_preds = np.load("npy/conv_final_preds.npy")
    tst_df = pd.read_csv("datasets_fin/sample_submission.csv")
except FileNotFoundError:
    print("오류: 3-모델 .npy 파일 또는 sample_submission.csv가 없습니다.")

# 2. [✅ 핵심] 0.9069점을 만든 '가중치'로 앙상블
# (이전에 0.9069점을 얻은 가중치 조합을 사용하세요. 예: B4:30, ViT:40, Conv:30)
print("0.9069점 앙상블 가중치로 확률 계산 중...")
image_ensemble_probs = (effnet_preds * 0.3) + (vit_preds * 0.4) + (convnext_preds * 0.3)

# 3. 예측 라벨 및 '최대 확률' 추출
pseudo_labels = np.argmax(image_ensemble_probs, axis=1)
max_probs = np.max(image_ensemble_probs, axis=1)

# 4. [✅ 핵심] '확증 편향' 방지를 위한 '높은 임계값'
CONFIDENCE_THRESHOLD = 0.999 # (99.9% 이상 확신하는 샘플만)

# 5. 임계값을 넘는 '확신' 샘플의 인덱스(indices) 추출
confident_indices = np.where(max_probs > CONFIDENCE_THRESHOLD)[0]

if len(confident_indices) == 0:
    print("🚨 경고: 99.9% 이상 확신하는 샘플이 0개입니다. THRESHOLD를 0.99로 낮춰보세요.")
else:
    print(f"✅ {len(confident_indices)}개의 '확신' 샘플(가짜 정답)을 찾았습니다.")

# 6. '가짜 정답' DataFrame 생성
pseudo_label_df = pd.DataFrame({
    'ID': tst_df.iloc[confident_indices]['ID'],
    'target': pseudo_labels[confident_indices]
})

# 7. '가짜 정답' CSV 저장
pseudo_label_df.to_csv("datasets_fin/pseudo_label.csv", index=False)
print("✅ 'pseudo_label.csv' 파일 저장 완료.")

--- 1단계: '가짜 정답(Pseudo-Label)' 생성 시작 ---
0.9069점 앙상블 가중치로 확률 계산 중...
✅ 1022개의 '확신' 샘플(가짜 정답)을 찾았습니다.
✅ 'pseudo_label.csv' 파일 저장 완료.
